In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'scipy'

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# add your code here - consider creating a new cell for each section of code
# Count ratings per user and per book (using ISBN to avoid title duplicates initially)
user_counts = df_ratings.groupby('user').size()
book_counts = df_ratings.groupby('isbn').size()

# Filter users with at least 200 ratings and books with at least 100 ratings
active_users = user_counts[user_counts >= 200].index
popular_books = book_counts[book_counts >= 100].index

# Filter ratings
df_ratings_filtered = df_ratings[df_ratings['user'].isin(active_users) & df_ratings['isbn'].isin(popular_books)]

# Merge with book titles
df_filtered = df_ratings_filtered.merge(df_books[['isbn', 'title']], on='isbn')

# Drop duplicates to ensure clean data
df_filtered = df_filtered.drop_duplicates(['user', 'title'])

# Create pivot table with titles as index
pivot_table = df_filtered.pivot_table(index='title', columns='user', values='rating').fillna(0)

# Convert to sparse matrix for efficiency
matrix = csr_matrix(pivot_table.values)

# Train KNN model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=6)
model_knn.fit(matrix)

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  # Find the book in the pivot table
  if book not in pivot_table.index:
    return [book, []]
  
  # Get the index of the book
  book_idx = pivot_table.index.get_loc(book)
  
  # Get neighbors
  distances, indices = model_knn.kneighbors([pivot_table.iloc[book_idx].values], n_neighbors=6)
  
  # Build recommendations list (exclude the first one as it's the book itself)
  recommendations = []
  for i in range(1, len(distances[0])):
    recommended_book = pivot_table.index[indices[0][i]]
    distance = float(distances[0][i])  # Convert to Python float
    recommendations.append([recommended_book, distance])
  
  # Reverse to get furthest first
  recommendations.reverse()
  
  return [book, recommendations]

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()